# Fadeel Khan
## Senior Design Pre-Processing and Feature Engineering

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Pre-Processing to Usable Form

We are including the mean, max and min values of each clinical trial. We intend to use the mean value as an indictor of the label. However, we are incorporating max and min to see if they have any correlation with the labels.

In [ ]:
def df_maker(testdf, type_domain):
  means = pd.DataFrame()
  maxes = pd.DataFrame()
  mins = pd.DataFrame()
  Jerk = pd.DataFrame()

  #group all rows by label of activity
  test = testdf.groupby(['label'])

  #add min and max values
  tester = test.mean()
  maxx = testdf.groupby(['label']).max()
  minn = testdf.groupby(['label']).min()

  # Renaming
  if type_domain == 'freq':
    tester = tester.rename(columns={"acc_x": "F_acc_x", "acc_y": "F_acc_y", "acc_z": "F_acc_z", "gyro_x": "F_gyro_x", "gyro_y": "F_gyro_y", "gyro_z": "F_gyro_z", "azimuth": "F_azimuth", "pitch": "F_pitch", "roll": "F_roll"})
    maxx = maxx.rename(columns={"acc_x": "Fmax_acc_x", "acc_y": "Fmax_acc_y", "acc_z": "Fmax_acc_z", "gyro_x": "Fmax_gyro_x", "gyro_y": "Fmax_gyro_y", "gyro_z": "Fmax_gyro_z", "azimuth": "Fmax_azimuth", "pitch": "Fmax_pitch", "roll": "Fmax_roll"})
    minn = minn.rename(columns={"acc_x": "Fmin_acc_x", "acc_y": "Fmin_acc_y", "acc_z": "Fmin_acc_z", "gyro_x": "Fmin_gyro_x", "gyro_y": "Fmin_gyro_y", "gyro_z": "Fmin_gyro_z", "azimuth": "Fmin_azimuth", "pitch": "Fmin_pitch", "roll": "Fmin_roll"})
    
    # Jerk
    dt = 150E-3 # 150ms
    avg_jerkk = tester/dt
    avg_jerkk = avg_jerkk.rename(columns={"F_acc_x": "jerk_Facc_x", "F_acc_y": "jerk_Facc_y", "F_acc_z": "jerk_Facc_z", "F_gyro_x": "jerk_Fgyro_x", "F_gyro_y": "jerk_Fgyro_y", "F_gyro_z": "jerk_Fgyro_z", "F_azimuth": "jerk_Fazimuth", "F_pitch": "jerk_Fpitch", "F_roll": "jerk_Froll"})
    avg_jerkk['F_JerkMagAcc'] = np.sqrt((avg_jerkk['jerk_Facc_x'])**2 + (avg_jerkk['jerk_Facc_y'])**2 + (avg_jerkk['jerk_Facc_z'])**2)
    avg_jerkk['F_JerkMagGyro'] = np.sqrt((avg_jerkk['jerk_Fgyro_x'])**2 + (avg_jerkk['jerk_Fgyro_y'])**2 + (avg_jerkk['jerk_Fgyro_z'])**2)
    avg_jerkk['F_JerkMagOrient'] = np.sqrt((avg_jerkk['jerk_Fazimuth'])**2 + (avg_jerkk['jerk_Fpitch'])**2 + (avg_jerkk['jerk_Froll'])**2)

  else:
    maxx = maxx.rename(columns={"acc_x": "max_acc_x", "acc_y": "max_acc_y", "acc_z": "max_acc_z", "gyro_x": "max_gyro_x", "gyro_y": "max_gyro_y", "gyro_z": "max_gyro_z", "azimuth": "max_azimuth", "pitch": "max_pitch", "roll": "max_roll"})
    minn = minn.rename(columns={"acc_x": "min_acc_x", "acc_y": "min_acc_y", "acc_z": "min_acc_z", "gyro_x": "min_gyro_x", "gyro_y": "min_gyro_y", "gyro_z": "min_gyro_z", "azimuth": "min_azimuth", "pitch": "min_pitch", "roll": "min_roll"})

    # Jerk
    dt = 150E-3 # 150ms
    avg_jerkk = tester/dt
    avg_jerkk = avg_jerkk.rename(columns={"acc_x": "jerk_acc_x", "acc_y": "jerk_acc_y", "acc_z": "jerk_acc_z", "gyro_x": "jerk_gyro_x", "gyro_y": "jerk_gyro_y", "gyro_z": "jerk_gyro_z", "azimuth": "jerk_azimuth", "pitch": "jerk_pitch", "roll": "jerk_roll"})
    avg_jerkk['JerkMagAcc'] = np.sqrt((avg_jerkk['jerk_acc_x'])**2 + (avg_jerkk['jerk_acc_y'])**2 + (avg_jerkk['jerk_acc_z'])**2)
    avg_jerkk['JerkMagGyro'] = np.sqrt((avg_jerkk['jerk_gyro_x'])**2 + (avg_jerkk['jerk_gyro_y'])**2 + (avg_jerkk['jerk_gyro_z'])**2)
    avg_jerkk['JerkMagOrient'] = np.sqrt((avg_jerkk['jerk_azimuth'])**2 + (avg_jerkk['jerk_pitch'])**2 + (avg_jerkk['jerk_roll'])**2)


  #add aggregated data into master data frame
  means = means.append(tester)
  mins = mins.append(minn)
  maxes = maxes.append(maxx)
  Jerk = Jerk.append(avg_jerkk)

  # Moving labels from index to a column
  means = means.reset_index(drop=False)
  maxes = maxes.reset_index(drop=False)
  mins = mins.reset_index(drop=False)
  Jerk = Jerk.reset_index(drop=False)

  maxes = maxes.drop(columns = ['label'])
  mins = mins.drop(columns = ['label'])
  Jerk = Jerk.drop(columns = ['label'])

  master = pd.concat([means,maxes,mins,Jerk], axis=1)
  return master

In [ ]:
# import necessary libraries
import pandas as pd
import os
import glob
import numpy as np  
  
# use glob to get all the csv files 
# in the folder
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))


#create list of all activity titles
master = pd.DataFrame()
inter = pd.DataFrame()
freq_inter = pd.DataFrame()


location = '/content/drive/Shareddrives/Team Three Seasons/Cycle 3/fall_data/annotated_data'
standinglist = (os.listdir(location))

for i in standinglist:
  freqs_magnitude = pd.DataFrame()
  filepath = location + '/' + i
  #convert file in testdf
  main_df = pd.read_csv(filepath)
  # Frequency Domain
  # This returns the fourier transform coeficients as complex numbers
  features = main_df.drop(columns = ['label'])
  for idx in features.columns:
    transformed_y = np.fft.fft(main_df[idx])
    # Take the absolute value of the complex numbers for magnitude spectrum
    freqs_magnitude[idx] = np.abs(transformed_y)

  freqs_magnitude['label'] = main_df['label']

  # Apppending
  inter = inter.append(df_maker(main_df, 'time'))
  freq_inter = freq_inter.append(df_maker(freqs_magnitude, 'freq'))
  freq_inter = freq_inter.drop(columns = ['label'])
  

master = pd.concat([inter,freq_inter], axis=1)

In [ ]:
master

,label,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,azimuth,pitch,roll,...,jerk_Facc_z,jerk_Fgyro_x,jerk_Fgyro_y,jerk_Fgyro_z,jerk_Fazimuth,jerk_Fpitch,jerk_Froll,F_JerkMagAcc,F_JerkMagGyro,F_JerkMagOrient
0,A1,-0.424800,9.773600,-1.077600,0.212400,-1.508800,1.878000,-16.325200,-40.876000,-22.086800,...,46.218350,597.712473,639.563804,384.046804,176.670224,508.787412,316.954640,129.646171,955.925733,624.929790
0,A13,-0.587333,7.016000,-5.809333,-2.942000,-0.300000,0.282667,-9.430667,-42.713333,-5.404000,...,77.273232,607.458987,204.247950,80.594612,99.351048,376.382095,253.698422,107.309184,645.925025,464.647180
0,A9,-0.915625,8.667500,-1.585000,-3.343750,2.906250,1.730625,44.330000,-14.691875,-22.371250,...,76.312784,579.532423,409.299234,305.781328,365.765230,190.777533,244.003642,122.925788,772.583919,479.289107
0,A6,-1.137222,9.206111,-1.877778,2.478889,-0.927222,0.717222,33.677778,-20.738889,-20.607778,...,66.236385,988.342112,155.709556,189.742159,378.345391,264.904271,229.752358,110.206428,1018.365201,515.854295
0,A6,-0.782143,9.217857,-2.095000,0.070714,-0.224286,-0.224286,37.022857,-20.415000,-19.660714,...,62.354511,1038.236586,151.494145,220.120862,385.362988,262.421005,231.590670,106.272171,1072.072235,520.580114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,F6,8.085714,2.160000,-0.842857,8.877143,16.080000,34.741429,-7.170000,-33.850000,9.330000,...,78.621263,686.146993,835.310350,1045.009796,335.944135,394.935071,228.464874,211.280806,1503.524709,566.593656
0,F6,4.796667,11.678333,2.155000,10.605000,63.521667,52.021667,6.103333,-42.198333,-0.310000,...,135.553952,422.683770,1189.725102,1712.880685,170.711816,309.773548,141.274887,291.134031,2127.925663,380.868441
0,F6,0.849091,7.896364,-2.800909,23.375455,-23.086364,5.517273,10.624545,-41.280909,-13.130000,...,114.166322,1033.419686,1203.022129,484.144915,208.706222,298.607830,253.350651,166.174331,1658.196245,443.747085
0,F6,8.943333,1.900000,-0.906667,36.240000,12.948333,40.926667,-18.021667,-36.875000,6.031667,...,34.907468,665.202144,548.082059,1227.056461,562.786133,437.956677,230.345097,198.272241,1499.518388,749.395186


In [ ]:
master.columns

Index(['label', 'acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z',
       'azimuth', 'pitch', 'roll', 'max_acc_x', 'max_acc_y', 'max_acc_z',
       'max_gyro_x', 'max_gyro_y', 'max_gyro_z', 'max_azimuth', 'max_pitch',
       'max_roll', 'min_acc_x', 'min_acc_y', 'min_acc_z', 'min_gyro_x',
       'min_gyro_y', 'min_gyro_z', 'min_azimuth', 'min_pitch', 'min_roll',
       'jerk_acc_x', 'jerk_acc_y', 'jerk_acc_z', 'jerk_gyro_x', 'jerk_gyro_y',
       'jerk_gyro_z', 'jerk_azimuth', 'jerk_pitch', 'jerk_roll', 'JerkMagAcc',
       'JerkMagGyro', 'JerkMagOrient', 'F_acc_x', 'F_acc_y', 'F_acc_z',
       'F_gyro_x', 'F_gyro_y', 'F_gyro_z', 'F_azimuth', 'F_pitch', 'F_roll',
       'Fmax_acc_x', 'Fmax_acc_y', 'Fmax_acc_z', 'Fmax_gyro_x', 'Fmax_gyro_y',
       'Fmax_gyro_z', 'Fmax_azimuth', 'Fmax_pitch', 'Fmax_roll', 'Fmin_acc_x',
       'Fmin_acc_y', 'Fmin_acc_z', 'Fmin_gyro_x', 'Fmin_gyro_y', 'Fmin_gyro_z',
       'Fmin_azimuth', 'Fmin_pitch', 'Fmin_roll', 'jerk_Facc_x', 'jerk_Facc_y',
    

In [ ]:
print('Shape of dataset is', master.shape, '\n')
master.head(5)

Shape of dataset is (200, 79) 



,label,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,azimuth,pitch,roll,...,jerk_Facc_z,jerk_Fgyro_x,jerk_Fgyro_y,jerk_Fgyro_z,jerk_Fazimuth,jerk_Fpitch,jerk_Froll,F_JerkMagAcc,F_JerkMagGyro,F_JerkMagOrient
0,A1,-0.424800,9.773600,-1.077600,0.212400,-1.508800,1.878000,-16.325200,-40.876000,-22.086800,...,46.218350,597.712473,639.563804,384.046804,176.670224,508.787412,316.954640,129.646171,955.925733,624.929790
0,A13,-0.587333,7.016000,-5.809333,-2.942000,-0.300000,0.282667,-9.430667,-42.713333,-5.404000,...,77.273232,607.458987,204.247950,80.594612,99.351048,376.382095,253.698422,107.309184,645.925025,464.647180
0,A9,-0.915625,8.667500,-1.585000,-3.343750,2.906250,1.730625,44.330000,-14.691875,-22.371250,...,76.312784,579.532423,409.299234,305.781328,365.765230,190.777533,244.003642,122.925788,772.583919,479.289107
0,A6,-1.137222,9.206111,-1.877778,2.478889,-0.927222,0.717222,33.677778,-20.738889,-20.607778,...,66.236385,988.342112,155.709556,189.742159,378.345391,264.904271,229.752358,110.206428,1018.365201,515.854295
0,A6,-0.782143,9.217857,-2.095000,0.070714,-0.224286,-0.224286,37.022857,-20.415000,-19.660714,...,62.354511,1038.236586,151.494145,220.120862,385.362988,262.421005,231.590670,106.272171,1072.072235,520.580114


In [ ]:
from pathlib import Path  
filepath = Path('/content/drive/Shareddrives/Team Three Seasons/Cycle 3/Machine Learning/master_dataset.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
master.to_csv(filepath) 